<a href="https://colab.research.google.com/github/Gowthamtj17/SEAMLESS-M4T-POWERED-SPEECH-TO-TEXT-WITH-GRADIO-UI.ipynb./blob/main/SEAMLESS_M4T_POWERED_SPEECH_TO_TEXT_WITH_GRADIO_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch==2.0.1

In [3]:
!pip install fairseq2==0.1 pydub
!git clone https://github.com/facebookresearch/seamless_communication.git
%cd seamless_communication
!git checkout 01c1042841f9bce66902eb2c7512dbdd71d42112
!pip install .

fatal: destination path 'seamless_communication' already exists and is not an empty directory.
/content/seamless_communication
HEAD is now at 01c1042 Update README.md
Processing /content/seamless_communication
  Preparing metadata (setup.py) ... done
  Created wheel for seamless_communication: filename=seamless_communication-0.1-py3-none-any.whl size=42638 sha256=1c99cd563e1225960116c2b1ede6a9768c77dbb24fa8c1b0850cab97fe9147c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-5edrptxl/wheels/62/83/f2/4a5a97ec76264de6b70d2d25bd272df84fe59cfc5216f662af
Successfully built seamless_communication
  Attempting uninstall: seamless_communication
    Found existing installation: seamless_communication 0.1
    Uninstalling seamless_communication-0.1:
      Successfully uninstalled seamless_communication-0.1


In [4]:
!pip install gradio==3.50.0
!pip install soundfile
!pip install librosa

In [5]:
import gradio as gr
import torch
from seamless_communication.models.inference import Translator
import librosa
import os
import soundfile as sf

In [6]:
# Initialize a Translator object with a multitask model and vocoder on the GPU
translator = Translator("seamlessM4T_large", vocoder_name_or_card="vocoder_36langs", device=torch.device("cuda:0"))


100%|██████████| 10.7G/10.7G [00:55<00:00, 206MB/s]
100%|██████████| 4.93M/4.93M [00:00<00:00, 62.6MB/s]
100%|██████████| 160M/160M [00:00<00:00, 233MB/s]


In [7]:
def downsample_audio(input_audio_file, output_audio_file, target_sr):
    audio_file, sr = librosa.load(input_audio_file, sr=None)
    audio_file_downsampled = librosa.resample(audio_file, orig_sr=sr, target_sr=16000)
    sf.write(output_audio_file, audio_file_downsampled, 16000)

In [8]:
def speech_to_text(input_audio_mic, input_audio_file, tgt_lang):
    # Check the input audio source and convert it to 16-bit integer format if needed
    if input_audio_mic is not None:
        input_audio_mic_path = input_audio_mic
    elif input_audio_file is not None:
        data, sr = sf.read(input_audio_file)
        sf.write("output.wav", data, sr, format='WAV', subtype='PCM_16')
        input_audio_file = "output.wav"
        input_audio_mic_path = input_audio_file
    else:
        return "Please provide an audio input."
    try:
        # Downsample the input audio file to 16000 Hz
        downsampled_audio_file = "downsampled_output.wav"
        downsample_audio(input_audio_mic_path, downsampled_audio_file, 16000)
        with open(downsampled_audio_file, "rb") as file:
            # Predict the translated text using seamless communication model
            translated_text, _, _ = translator.predict(file.name, "s2tt", tgt_lang)
            # Return the translated text as output
            return translated_text
    except Exception as e:
        # Return an error message if any exception occurs during translation
        return f"Error translating {input_audio_mic_path}: {e}"

In [9]:
# Create UI components for audio input, target language selection and translated text output
audio_input_mic = gr.Audio(source="microphone", type="filepath", label="Input Audio from Microphone")
audio_input_file = gr.Audio(source="upload", type="filepath", label="Input Audio from File")
tgt_lang = gr.Dropdown(choices=[("English", "eng"), ("Tamil", "tam")], label="Target Language")
translated_text_output = gr.Textbox(label="Translated Text")

In [10]:
# Create an interface object with speech_to_text function and UI components
demo = gr.Interface(
    fn=speech_to_text,
    inputs=[audio_input_mic, audio_input_file, tgt_lang],
    outputs=translated_text_output,
    title="SEAMLESS M4T-POWERED SPEECH TO TEXT WITH GRADIO UI",
    description="Translate Audio",
)

In [11]:
# Launch the interface in a web browser
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://243cf81fa471b42989.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
